In [1]:
config = {
    "pretrained_model": "bert-base-cased",
    "tokenizer": "bert-base-cased",
    "max_seq_length": 512,
    "batch_size": 10,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cuda",
    "gpu_ids": "1",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 4,
    "is_multilabel": False,
    "valid_metric": "macro_f1",
    "model_save_dir": "../checkpoints/bert_cased_512_biocaster_4cate_20200903/",
    "patience": 4,
}

In [17]:
import sys
sys.path.append("../")
from beta_nlp.utils.data_util import biocaser2text,biocaster2df
data_file = "/home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml"
data_df = biocaster2df(data_file)
data_df["source"] = "Biocaster"
data_df.head(3)

parse biocaser data from /home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml, docs number:1003, lablels number:1003


,docs,labels,source
0,\nBird Flu Outbreak Drill Spooks Manitoba Town...,0,Biocaster
1,\nTyphoid outbreak in Agusan del Sur town unde...,3,Biocaster
2,\n Typhoid Outbreak In Central Nepal November...,3,Biocaster


In [18]:
data_df.labels = data_df.labels.apply(lambda x: "01" if x==0 else "10")
data_df.docs = data_df.docs.apply(lambda x: x.replace("\n",". ").replace("\r",""))

In [19]:
data_df.head()

,docs,labels,source
0,. Bird Flu Outbreak Drill Spooks Manitoba Town...,01,Biocaster
1,. Typhoid outbreak in Agusan del Sur town unde...,10,Biocaster
2,. Typhoid Outbreak In Central Nepal November...,10,Biocaster
3,. Chlorine truck crash shuts Turnpike exit. Po...,01,Biocaster
4,. Man dies from rare anthrax bug . Christopher...,10,Biocaster


In [20]:
from sklearn.utils import shuffle
import random
import numpy as np
random.seed(2020)
np.random.seed(2020)
data_df = shuffle(data_df).reset_index()
data_df["flag"]=None
train_index = int(len(data_df.index)*0.8)
valid_index = train_index + round(len(data_df.index)*0.1)
train_set = data_df.iloc[:train_index]
dev_set = data_df.iloc[train_index:valid_index]
test_set = data_df.iloc[valid_index:]
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [21]:
train_set.to_csv("../covid19-classification/hedwig-data/datasets/Biocaster/train.tsv",
                columns=["labels","docs"],
                header=False,
                index=False,
                sep="\t")
dev_set.to_csv("../covid19-classification/hedwig-data/datasets/Biocaster/dev.tsv",
                columns=["labels","docs"],
                header=False,
                index=False,
                sep="\t")
test_set.to_csv("../covid19-classification/hedwig-data/datasets/Biocaster/test.tsv",
                columns=["labels","docs"],
                header=False,
                index=False,
                sep="\t")

In [4]:
from sklearn.utils import shuffle
import random
import numpy as np
random.seed(2020)
np.random.seed(2020)
data_df = shuffle(data_df).reset_index()
data_df["flag"]=None
train_index = int(len(data_df.index)*0.8)
valid_index = train_index + round(len(data_df.index)*0.1)
data_df.iloc[:train_index]["flag"]="train"
data_df.iloc[train_index:valid_index]["flag"]="valid"
data_df.iloc[valid_index:]["flag"]="test"

<ipython-input-4-f9ee4dff7589>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[:train_index]["flag"]="train"
<ipython-input-4-f9ee4dff7589>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[train_index:valid_index]["flag"]="valid"
<ipython-input-4-f9ee4dff7589>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [4]:
train_set = data_df[data_df["flag"]=="train"]
dev_set = data_df[data_df["flag"]=="valid"]
test_set = data_df[data_df["flag"]=="test"]

In [5]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [6]:
from beta_nlp.models.bert_cls import BertModel
cls = BertModel(config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [7]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [8]:
cls.train(train_set,dev_set)

Evaluating:  10%|█         | 1/10 [00:00<00:01,  7.21it/s]

Execute [train_an_epoch] method costing 31652.89 ms
[Epoch 0] loss: 89.15447789430618



Evaluating: 100%|██████████| 10/10 [00:01<00:00,  7.33it/s]
/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Execute [eval] method costing 1369.24 ms
------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.81     |
|  1 | macro_precision | 0.387203 |
|  2 | macro_recall    | 0.430589 |
|  3 | macro_f1        | 0.407744 |
|  4 | micro_precision | 0.81     |
|  5 | micro_recall    | 0.81     |
|  6 | micro_f1        | 0.81     |
+----+-----------------+----------+
------------------------------------------------------------


Evaluating:  10%|█         | 1/10 [00:00<00:01,  6.93it/s]

...

Execute [eval] method costing 1452.69 ms
------------------------------------------------------------
[Epoch 8]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.84     |
|  1 | macro_precision | 0.557852 |
|  2 | macro_recall    | 0.536759 |
|  3 | macro_f1        | 0.531226 |
|  4 | micro_precision | 0.84     |
|  5 | micro_recall    | 0.84     |
|  6 | micro_f1        | 0.84     |
+----+-----------------+----------+
------------------------------------------------------------


Evaluating:  10%|█         | 1/10 [00:00<00:01,  6.96it/s]

Execute [train_an_epoch] method costing 34507.08 ms
[Epoch 9] loss: 10.914065071498044



Epoch: 100%|██████████| 10/10 [06:04<00:00, 36.41s/it]

Execute [eval] method costing 1442.44 ms


In [9]:
cls.test(test_set)

Evaluating: 100%|██████████| 11/11 [00:01<00:00,  7.60it/s]

Execute [eval] method costing 1450.58 ms


{'accuracy': 0.7920792079207921,
 'macro_precision': 0.48461538461538456,
 'macro_recall': 0.5086050724637681,
 'macro_f1': 0.48376623376623373,
 'micro_precision': 0.7920792079207921,
 'micro_recall': 0.7920792079207921,
 'micro_f1': 0.7920792079207921}

In [10]:
test_set.iloc[1:10]

,index,docs,labels,source,flag
903,482,\n Plague in the Democratic Republic of the C...,3,Biocaster,test
904,245,\n TNT workers walk off job over IR laws\n Ap...,0,Biocaster,test
905,562,\n\nEarly signs of meningitis found \nThe char...,0,Biocaster,test
906,792,\nAirlines take new steps to save fuel \n \n20...,0,Biocaster,test
907,708,\n Cat parasite is killing otters \n \n \n By...,0,Biocaster,test
908,358,\n Iranian Attacks on PKK Underway in N. Iraq...,0,Biocaster,test
909,715,\n Mans canal death is unexplained\n \n A man...,0,Biocaster,test
910,927,\n Anthrax affects 20 people in W Bengal vill...,3,Biocaster,test
911,667,\nNYT-01-31-96 1524EST\nBC-RESEARCH-SHIP-BOS N...,0,Biocaster,test


In [11]:
cls.predict(test_set.iloc[1:10])

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  7.94it/s]

Execute [predict] method costing 147.44 ms


array([3, 0, 3, 0, 0, 0, 3, 3, 0])

### 